In [ ]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
import skimage.io
# ceļu norādīt nevajag, ja pakotne ir instalēta
documentpath=os.getcwd()
libpath=os.path.abspath(os.path.join(documentpath, os.pardir))
sys.path.append(libpath)
from sudrabainiemakoni.cloudimage import CloudImage
from sudrabainiemakoni.cloudimage import CloudImagePair
from sudrabainiemakoni.cloudimage import WebMercatorImage
from sudrabainiemakoni import plots

In [ ]:
# katalogs ar piemēra failiem
sample_directory = 'SampleData'
# katalogs ar rezultātu failiem
results_directory = 'SampleResults'
# pirmais attēls
case1 = 'js_202106210100'
# otrais attēls
case2 = ''

In [ ]:
# izveidojam divus CloudImage tipa objektus, katru savam attēlam
# Ielasam no faila, pirms tam referencētus attēlus
cldim1 = CloudImage.load(f'{results_directory}/{case1}.proj')
cldim2 = CloudImage.load(f'{results_directory}/{case2}.proj')
print(cldim1)
print(cldim2)

In [ ]:
# izveidojam CloudImagePair objektu, kas darbojas ar abiem attēliem
imagepair = CloudImagePair(cldim1, cldim2)

In [ ]:
# ielasam atbilstības punktu pikseļu koordinātes
imagepair.LoadCorrespondances(f"{sample_directory}/{case1}_{case2}.txt")

### Atrodam augstuma punktus

Atrodam kopīgo punktu koordinātes (platums, garums, augstums) (*llh*), attālumu starp stariem uz kopīgajiem punktiem (metros) (*rayminimaldistance*), (*z_intrinsic_error*) - potenciālā kļūda km/px (jo mazāka jo labāk - kopīgie punkti atrodas uz garāka epilīniju nogriežņa), *valid* - iezīme, vai punkts derīgs atbilstoši kritētijiem

In [ ]:
llh, rayminimaldistance, z_intrinsic_error, valid = imagepair.GetHeightPoints(*imagepair.correspondances)

In [ ]:
# sagatavojam punktu datus tabulas formā
df_points =pd.DataFrame({'lat':llh[0], 'lon':llh[1], 'z':llh[2], 'Rdist':rayminimaldistance, 'zerr':z_intrinsic_error, 'valid':valid})
df_points

### Izveidojam punktu attēlus

In [ ]:
# izveidojam punktiem atbilstošos epilīniju nogriežņus augstumu intervālam 75 - 90 km
z1, z2 = 75, 90
# epilīnijas pirmajā attēlā, kas atbilst punktiem otrajā attēlā
epilines = imagepair.GetEpilinesAtHeightInterval([z1,z2],imagepair.correspondances[1], False)
# izvadām punktu attēlus
plots.PlotValidHeightPoints(cldim1.imagearray,epilines,imagepair.correspondances[0] , llh[2], valid,
                            filename = f"{results_directory}/heights_points_{case1}_{case2}.jpg")
plots.PlotValidHeightPoints(cldim1.imagearray,epilines,imagepair.correspondances[0] , llh[2], None,
                            filename = f"{results_directory}/heights_points_{case1}_{case2}_all.jpg")


In [ ]:
# epilīnijas otrajā attēlā, kas atbilst punktiem pirmajā attēlā
epilines = imagepair.GetEpilinesAtHeightInterval([z1,z2],imagepair.correspondances[0], True)
# izvadām punktu attēlus
plots.PlotValidHeightPoints(cldim2.imagearray,epilines,imagepair.correspondances[1] , llh[2], valid,
                            filename = f"{results_directory}/heights_points_{case2}_{case1}.jpg")
plots.PlotValidHeightPoints(cldim2.imagearray,epilines,imagepair.correspondances[1] , llh[2], None,
                            filename = f"{results_directory}/heights_points_{case2}_{case1}_all.jpg")

### Izveidojam interpolētu augstumu sadalījumu  

Veidosim to ar *webmerc* tipa objekta palīdzību uz tā paša režģa, uz kura projicējām attēlus  
Lietosim projicētos punktus no pirmā attēla

In [ ]:
# projicēšanas apgabals un izšķirtspēja
lonmin, lonmax, latmin, latmax, horizontal_resolution_km = 15, 35, 57, 64, 0.5

In [ ]:
# sagatavojam projicēšanas objektu
webmerc = WebMercatorImage(cldim1, lonmin, lonmax, latmin, latmax, horizontal_resolution_km)
webmerc2 = WebMercatorImage(cldim2, lonmin, lonmax, latmin, latmax, horizontal_resolution_km)

In [ ]:
# veidojam interpolēto augstumu sadalījumu ar *kriging* palīdzību, no derīgajiem punktiem
heightgrid = webmerc.PrepareHeightMap(llh[1][valid],llh[0][valid],llh[2][valid])

In [ ]:
# varam noglabāt augstumu masīvu failā
np.save(f"{results_directory}/heightmap_{case1}_{case2}.npy", heightgrid)

In [ ]:
# attēls ātrai vizuālai kontrolei
fig, ax=plt.subplots(figsize=(20,10))
cs=ax.imshow(heightgrid)
fig.colorbar(cs)
plt.show()

In [ ]:
# augstumu sadalījums uz kartes vizuālai kontrolei
map_lonmin, map_lonmax, map_latmin, map_latmax = 15,33,56,64
pp=[[cldim1.location.lon.value, cldim1.location.lat.value]]
plots.PlotReferencedImages(webmerc, [heightgrid],  camera_points=pp,
                               outputFileName=None,
                               lonmin=map_lonmin, lonmax=map_lonmax, latmin=map_latmin, latmax=map_latmax,
                               showplot=True,
                               alpha=0.8)


### Projicējam abus attēlus ņemot vērā augstumu sadalījumu

Šeit var lietot visas *Projicet* piemēra metodes, ko darīt ar projicētiem attēliem

In [ ]:
# izveidojam abus projicētos attēlus
webmerc.prepare_reproject_from_camera(heightgrid/1000)
projected_image_hght=webmerc.Fill_projectedImageMasked()
webmerc2.prepare_reproject_from_camera(heightgrid/1000)
projected_image_hght2=webmerc2.Fill_projectedImageMasked()

Izveidojam abus projicētos attēlus atsevišķi un arī kopā uz kartes pamatnes fona

In [ ]:
# novērotāju punkti
p1=[cldim1.location.lon.value, cldim1.location.lat.value]
p2=[cldim2.location.lon.value, cldim2.location.lat.value]
pp=[p1,p2]
# kopīgs attēls
plots.PlotReferencedImages(webmerc, [projected_image_hght, projected_image_hght2],
                           camera_points=pp,
                           showplot=True,
                           outputFileName=None) #f'{results_directory}/georeferenced_{case1}_{case2}_mapview.jpg')

In [ ]:
# atsevišķie attēli
plots.PlotReferencedImages(webmerc, [projected_image_hght],camera_points=pp[0:1],
                           outputFileName=f'{results_directory}/georeferenced_{case1}_mapview.jpg')
plots.PlotReferencedImages(webmerc, [projected_image_hght2],camera_points=pp[1:2],
                           outputFileName=f'{results_directory}/georeferenced_{case2}_mapview.jpg')